In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('../../data/train_airbnb_berlin.csv', delimiter = ',')

In [5]:
data.head()

'''
'Listing ID' : useless, take out of data
'Listing Name' : useless, take out of data
'Host ID' : transform feature to number of listings
'Host Name' : useless, take out of data (except if we can separate by gender --> new feature gender of host)
'Host Since' : transform date into a continuous number
'Host Response Time': build 4 discrete values + unknown for NaNs
'Host Response Rate': transform percentage to number + treat NaNs
'Is Superhost' : transform to binary 0, 1
'neighbourhood' : tranform classes to numbers (not sure, think about it)
'Neighbourhood Group' : transform classes to numbers, but is this feature usefull ??
'City' : useless, take out of data
'Postal Code' : leave as is ///////// numeric categorical data -> we could use an embedding (126-long vector with a 1 for right postal code)
'Country' : useless, take out of data
'Country Code' : useless, take out of data
'Latitude' : keep as is
'Longitude' : keep as is + we could think of a feature combining Latitude and Longitude
'Is Exact Location' : transform to binary
'Property Type' : transform classes to numbers
'Room Type' : transform to 3 discrete variables
'Accomodates' : leave as is
'Bathrooms' : leave as is
'Bedrooms' : leave as is
'Beds' : leave as is
'Square Feet' : take out of data because 98% of NaNs
'Guests Included' : leave as is
'Min Nights' : leave as is
'Reviews' : leave as is (number of reviews)
'First Review' : transform date into a continuous number
'Last Review' : transform date into a continuous number
'Overall Rating' : leave as is
'Accuracy Rating' : leave as is
'Cleanliness Rating' : leave as is
'Checkin Rating' : leave as is
'Communication Rating' : leave as is
'Location Rating' : leave as is
'Value Rating' : leave as is
'Instant Bookable' : transform to binary
'Business Travel Ready' : transform to binary
'Price' : leave as is
'''

,Listing ID,Listing Name,Host ID,Host Name,Host Since,Host Response Time,Host Response Rate,Is Superhost,neighbourhood,Neighborhood Group,...,Overall Rating,Accuracy Rating,Cleanliness Rating,Checkin Rating,Communication Rating,Location Rating,Value Rating,Instant Bookable,Business Travel Ready,Price
0,19665213.0,*,156079597.0,Maximilian,2016-01-20,NaN,NaN,f,Prenzlauer Berg,Pankow,...,100.0,10.0,10.0,10.0,10.0,9.0,10.0,t,f,26.0
1,6436842.0,*,5302290.0,Dulie,2013-04-07,NaN,NaN,f,Pankow,Pankow,...,90.0,9.0,9.0,10.0,10.0,9.0,10.0,f,f,41.0
2,10559468.0,*,59151456.0,Geank,2016-02-07,NaN,NaN,f,Prenzlauer Berg,Pankow,...,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f,f,50.0
3,27215482.0,*,193452785.0,Alix,2018-06-26,NaN,NaN,f,Friedrichshain,Friedrichshain-Kreuzberg,...,100.0,10.0,10.0,10.0,10.0,10.0,9.0,f,f,50.0
4,27287546.0,*,205870244.0,Lurina,2013-05-16,within a few hours,92%,t,Prenzlauer Berg,Pankow,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t,f,55.0


In [6]:
data.shape

(15692, 39)

In [8]:
data.isnull().sum()/data.shape[0]

Listing ID               0.000064
Listing Name             0.003441
Host ID                  0.000000
Host Name                0.001402
Host Since               0.001338
Host Response Time       0.450867
Host Response Rate       0.450867
Is Superhost             0.001466
neighbourhood            0.000000
Neighborhood Group       0.000000
City                     0.000064
Postal Code              0.014593
Country Code             0.000000
Country                  0.000000
Latitude                 0.000000
Longitude                0.000000
Is Exact Location        0.000000
Property Type            0.000000
Room Type                0.000000
Accomodates              0.000000
Bathrooms                0.000892
Bedrooms                 0.000319
Beds                     0.000510
Square Feet              0.980691
Guests Included          0.000000
Min Nights               0.000000
Reviews                  0.000000
First Review             0.172381
Last Review              0.172445
Overall Rating

In [9]:
data.isnull().sum()

Listing ID                   1
Listing Name                54
Host ID                      0
Host Name                   22
Host Since                  21
Host Response Time        7075
Host Response Rate        7075
Is Superhost                23
neighbourhood                0
Neighborhood Group           0
City                         1
Postal Code                229
Country Code                 0
Country                      0
Latitude                     0
Longitude                    0
Is Exact Location            0
Property Type                0
Room Type                    0
Accomodates                  0
Bathrooms                   14
Bedrooms                     5
Beds                         8
Square Feet              15389
Guests Included              0
Min Nights                   0
Reviews                      0
First Review              2705
Last Review               2706
Overall Rating            2962
Accuracy Rating           2971
Cleanliness Rating        2970
Checkin 

In [36]:
len(data["neighbourhood"].unique().tolist())

63

In [37]:
len(data["Postal Code"].unique().tolist())

186

In [39]:
len(data["Neighborhood Group"].unique().tolist())

12

In [40]:
len(data["Property Type"].unique().tolist())

16

In [41]:
len(data["Room Type"].unique().tolist())

3

In [42]:
data["Room Type"].unique().tolist()

['Private room', 'Entire home/apt', 'Shared room']

In [33]:
data_copy = data

In [34]:
data_copy = data_copy.drop(columns=["Listing ID", "Listing Name", "Country", "City", "Country Code", "Square Feet"])

In [35]:
len(data_copy.columns.tolist())

33

In [44]:
data["Accomodates"].unique().tolist()

['2', '3', '4', '1', '6', '10', '9', '5', '8', '7', '12', '*', '16']

In [45]:
data["Bathrooms"].unique().tolist()

['1.0', '2.0', '1.5', '2.5', nan, '0.5', '*', '3.0', '0.0']

In [46]:
data["Bedrooms"].unique().tolist()

['1.0', '2.0', '0.0', '3.0', '4.0', '5.0', '*', nan]

In [47]:
data["Beds"].unique().tolist()

['1.0',
 '2.0',
 '4.0',
 '8.0',
 '6.0',
 '5.0',
 '3.0',
 '7.0',
 nan,
 '9.0',
 '10.0',
 '*',
 '0.0']

In [48]:
data["Guests Included"].unique().tolist()

['1', '2', '4', '5', '3', '6', '8', '*']

In [49]:
data["Min Nights"].unique().tolist()

['2',
 '7',
 '1',
 '6',
 '5',
 '4',
 '*',
 '3',
 '20',
 '14',
 '21',
 '9',
 '25',
 '10',
 '8',
 '60',
 '12',
 '30',
 '28',
 '90',
 '180',
 '13',
 '61',
 '15',
 '62']

In [63]:
data["City"].unique().tolist()

['Berlin', '*', nan]

In [53]:
for feature in data_copy.columns.tolist():
    unique_list = data_copy[feature].unique().tolist()
    if len(unique_list) < 100:
        print(f'{feature} (len = {len(unique_list)}) : \n {unique_list} \n')

Host Response Time (len = 5) : 
 [nan, 'within a few hours', 'within an hour', 'within a day', 'a few days or more'] 

Host Response Rate (len = 35) : 
 [nan, '92%', '100%', '80%', '71%', '83%', '98%', '78%', '89%', '50%', '90%', '70%', '*', '20%', '97%', '91%', '88%', '75%', '93%', '0%', '67%', '33%', '40%', '58%', '86%', '60%', '95%', '94%', '96%', '82%', '63%', '99%', '87%', '25%', '30%'] 

Is Superhost (len = 3) : 
 ['f', 't', nan] 

neighbourhood (len = 63) : 
 ['Prenzlauer Berg', 'Pankow', 'Friedrichshain', 'Mariendorf', 'Wilmersdorf', 'Kreuzberg', 'Moabit', 'NeukÃ¶lln', 'Alt-Treptow', 'SchÃ¶neberg', 'Reinickendorf', 'Mitte', 'Karlshorst', 'WeiÃ\x9fensee', 'Friedenau', 'Hansaviertel', 'Wedding', 'Charlottenburg', 'Steglitz', 'Baumschulenweg', 'Alt-HohenschÃ¶nhausen', 'Rummelsburg', 'Lichterfelde', 'OberschÃ¶neweide', 'Britz', 'Lichtenberg', 'Grunewald', 'Altglienicke', 'Westend', 'Tempelhof', 'Marzahn', 'Mahlsdorf', 'Adlershof', 'NiederschÃ¶nhausen', 'Tegel', 'Rahnsdorf', 'SchmÃ¶

In [58]:
data_pp = data

In [59]:
data_pp = data_pp.drop(columns=["Listing ID", "Listing Name", "City", "Country Code", "Country", "Square Feet",
                                "First Review", "Last Review", "Business Travel Ready"])

In [60]:
data_pp

,Host ID,Host Name,Host Since,Host Response Time,Host Response Rate,Is Superhost,neighbourhood,Neighborhood Group,Postal Code,Latitude,...,Reviews,Overall Rating,Accuracy Rating,Cleanliness Rating,Checkin Rating,Communication Rating,Location Rating,Value Rating,Instant Bookable,Price
0,156079597.0,Maximilian,2016-01-20,NaN,NaN,f,Prenzlauer Berg,Pankow,10437.0,52.54652,...,6,100.0,10.0,10.0,10.0,10.0,9.0,10.0,t,26.0
1,5302290.0,Dulie,2013-04-07,NaN,NaN,f,Pankow,Pankow,13187.0,52.56512,...,6,90.0,9.0,9.0,10.0,10.0,9.0,10.0,f,41.0
2,59151456.0,Geank,2016-02-07,NaN,NaN,f,Prenzlauer Berg,Pankow,10439.0,52.54741,...,2,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f,50.0
3,193452785.0,Alix,2018-06-26,NaN,NaN,f,Friedrichshain,Friedrichshain-Kreuzberg,10245,52.50958,...,4,100.0,10.0,10.0,10.0,10.0,10.0,9.0,f,50.0
4,205870244.0,Lurina,2013-05-16,within a few hours,92%,t,Prenzlauer Berg,Pankow,10405.0,52.52995,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t,55.0
5,90250336.0,Zan,2016-08-22,within an hour,100%,t,Mariendorf,Tempelhof - SchÃ¶neberg,*,52.44826,...,10,99.0,10.0,9.0,10.0,10.0,10.0,10.0,t,39.0
6,5947005.0,Meler,2013-03-13,within a day,80%,f,Wilmersdorf,Charlottenburg-Wilm.,*,52.49453,...,5,97.0,10.0,10.0,10.0,10.0,10.0,10.0,f,94.0
7,19435959.0,Alise,2015-04-30,within an hour,100%,f,Kreuzberg,Friedrichshain-Kreuzberg,10997,52.50021,...,14,94.0,9.0,9.0,10.0,10.0,10.0,10.0,t,73.0
8,151814038.0,Ologfi,2017-09-11,NaN,NaN,f,Kreuzberg,Friedrichshain-Kreuzberg,10961.0,52.49598,...,13,93.0,9.0,10.0,9.0,9.0,10.0,9.0,f,100.0
9,72266241.0,Aselgiiud,2016-05-06,within an hour,100%,f,Moabit,Mitte,10553.0,52.53145,...,2,100.0,10.0,9.0,10.0,10.0,9.0,10.0,f,50.0


In [69]:
s_latitude = data["Latitude"].sum()
s_longitude = data["Longitude"].sum()

print(s_latitude, s_longitude)

823985.248243 210387.8872


In [70]:
barycentre_lat = s_latitude/len(data)
barycentre_long = s_longitude/len(data)

In [71]:
print(barycentre_lat, barycentre_long)

52.50989346437675 13.407334132041804


In [72]:
data_pp["Distance from barycenter"] = np.sqrt((data_pp["Latitude"] - barycentre_lat)**2
                                              + (data_pp["Longitude"] - barycentre_long)**2)

In [75]:
data_pp["Distance from barycenter"]

0        0.038126
1        0.057177
2        0.041558
3        0.044107
4        0.021685
5        0.061646
6        0.080788
7        0.029444
8        0.019783
9        0.092773
10       0.102839
11       0.025698
12       0.047825
13       0.054744
14       0.080158
15       0.034150
16       0.120949
17       0.063137
18       0.049164
19       0.043480
20       0.080247
21       0.022817
22       0.073181
23       0.067413
24       0.023306
25       0.030276
26       0.076954
27       0.092282
28       0.022766
29       0.038259
           ...   
15662    0.022853
15663    0.049809
15664    0.107592
15665    0.043772
15666    0.084099
15667    0.033280
15668    0.040485
15669    0.038605
15670    0.041648
15671    0.027656
15672    0.026584
15673    0.091746
15674    0.028876
15675    0.018229
15676    0.046198
15677    0.049303
15678    0.035021
15679    0.066806
15680    0.051104
15681    0.226707
15682    0.032923
15683    0.042340
15684    0.043755
15685    0.072158
15686    0